<a href="https://colab.research.google.com/github/liuwang2000/colabScript/blob/main/notebooks/predict_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM prediction with Prime

This tutorial demonstrates how to predict the OGT of a protein using a pretrained model from the Prime model.

We provide:

- The sequences, a FASTA file.

Goals
Obtain an predicted TM for each sequence.


## Import the necessary libraries and modules.

In [2]:
# 克隆仓库（如果尚未克隆）
!git clone https://github.com/ai4protein/Pro-Prime.git
%cd Pro-Prime/notebooks
# 步骤1：挂载Google Drive（每次重启后需重新挂载）
from google.colab import drive
drive.mount('/content/drive')

# 步骤2：安装依赖
!pip install torch==2.4
!pip install torch_xla==2.4
!pip install biopython==1.81

Cloning into 'Pro-Prime'...
remote: Enumerating objects: 1244, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1244 (delta 43), reused 36 (delta 16), pack-reused 1173 (from 1)
Receiving objects: 100% (1244/1244), 67.98 MiB | 31.48 MiB/s, done.
Resolving deltas: 100% (291/291), done.
Updating files: 100% (814/814), done.
/content/Pro-Prime/notebooks/Pro-Prime/notebooks
Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 476.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 471.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from Bio import SeqIO
import os # Import the os modul
from tqdm.notebook import tqdm
from transformers import AutoModel

## Prepare data path

In [7]:
sequence_file = "/content/drive/MyDrive/test.fasta"

## Load model and Tokenizer

In [5]:
!huggingface-cli login # 确保登录成功

model_path = "AI4Protein/Prime_690M"  # 再次确认模型路径
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Remove `from_flax=True` as the model might not be in Flax format
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

model.eval()
model = model.to(device)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `my_api_key` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

configuration_proprime.py:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/Prime_690M:
- configuration_proprime.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenization_proprime.py:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/Prime_690M:
- tokenization_proprime.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

## Prediction

In [10]:
import pandas as pd
from Bio import SeqIO
from IPython.display import FileLink

tm = []
sequence_names = []  # 用于存储序列名称的列表

with torch.no_grad():
    for record in tqdm(list(SeqIO.parse(sequence_file, "fasta"))):
        sequence = str(record.seq)
        sequence_names.append(record.id)  # 获取序列名称并添加到列表
        tokenied_results = tokenizer(sequence, return_tensors="pt")
        input_ids = tokenied_results.input_ids.to(device)
        attention_mask = tokenied_results.attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).predicted_values
        tm.append(logits.item())

# 创建 pandas DataFrame
df = pd.DataFrame({'sequence_names': sequence_names, 'Tm': tm})

# 将 DataFrame 保存为 CSV 文件
csv_file_path = 'tm_predictions.csv'  # 指定文件名
df.to_csv(csv_file_path, index=False)  # 保存为 CSV 文件，不包含索引


  0%|          | 0/15 [00:00<?, ?it/s]

/content/Pro-Prime/notebooks/Pro-Prime/notebooks/tm_predictions.csv

In [11]:
# 指定保存路径
save_path = '/content/drive/MyDrive/tm_predictions.csv'  # 修改保存路径

# 将 DataFrame 保存为 CSV 文件
df.to_csv(save_path, index=False)  # 保存到指定路径

print(f"文件已保存到: {save_path}")  # 打印保存路径信息

文件已保存到: /content/drive/MyDrive/tm_predictions.csv
